# リスト内包表記とドット演算子の速度比較

In [1]:
using MyTools, BenchmarkTools

In [2]:
w = randn(1000)   #Soft-Thresholding適用対象の配列
w2 = zeros(1000)  #結果格納用（メモリ確保の時間を除くため）
l = 0.1           #Soft-Thresholdingの閾値
w'

1×1000 LinearAlgebra.Adjoint{Float64,Array{Float64,1}}:
 -1.02492  -1.89334  -0.622621  1.75002  …  -0.305902  -0.180492  1.36293

**リスト内包表記でSoft-Thresholding1**

In [3]:
@btime w2 .= [wi > l ? (wi - l) : wi < -l ? (wi + l) : 0.0 for wi in w]
w2'

  100.300 μs (3908 allocations: 68.98 KiB)


1×1000 LinearAlgebra.Adjoint{Float64,Array{Float64,1}}:
 -0.924921  -1.79334  -0.522621  …  0.0  -0.205902  -0.0804923  1.26293

**リスト内包表記でSoft-Thresholding2**

In [4]:
@btime w2 .= [abs(wi) > l ? sign(wi) * (abs(wi) - l) : 0.0 for wi in w]
w2'

  97.899 μs (4688 allocations: 81.17 KiB)


1×1000 LinearAlgebra.Adjoint{Float64,Array{Float64,1}}:
 -0.924921  -1.79334  -0.522621  …  0.0  -0.205902  -0.0804923  1.26293

**ドット演算子でSoft-Thresholding1**

In [5]:
@btime @. w2 = (w > l) * (w - l) + (w < -l) * (w + l)
w2'

  5.333 μs (16 allocations: 368 bytes)


1×1000 LinearAlgebra.Adjoint{Float64,Array{Float64,1}}:
 -0.924921  -1.79334  -0.522621  …  0.0  -0.205902  -0.0804923  1.26293

**ドット演算子でSoft-Thresholding2**

In [6]:
@btime @. w2 = (abs(w) > l) * sign(w) * (abs(w) - l)
w2'

  3.112 μs (12 allocations: 240 bytes)


1×1000 LinearAlgebra.Adjoint{Float64,Array{Float64,1}}:
 -0.924921  -1.79334  -0.522621  1.65002  …  -0.205902  -0.0804923  1.26293